In [1]:
import os
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim

### Load In the File and Initially Examine

In [153]:
# Define the folder containing the parquet files
folder_path = '../Data_Raw/project_tables'

# List all parquet files in the folder
parquet_files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]

# Read and concatenate all parquet files into one dataframe
df = pd.concat([pd.read_parquet(os.path.join(folder_path, f)) for f in parquet_files], ignore_index=True)

# Display the first few rows of the combined dataframe
df.head()

,project_number,project_link,project_type,project_location,project_description,last_reviewed,last_reviewed_meeting,review_type
0,OG 25-038,/records-research/project-search/og-25-038-0,Commercial,"[1023 - 1029 31st Street, NW, United States]",Sign - Canal House,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix)
1,OG 25-037,/records-research/project-search/og-25-037-0,Georgetown,"[3700 O Street, NW, United States]",Shaw Field lights,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix)
2,OG 25-036,/records-research/project-search/og-25-036-0,Residence,"[3321 Q Street, NW, United States]",Egress window well on west elevation,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix)
3,OG 25-034,/records-research/project-search/og-25-034-0,Residence,"[3306 O Street, NW, United States]",Shutter and door replacements,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix)
4,OG 25-033,/records-research/project-search/og-25-033-0,Residential,"[3114 R Street, NW, United States]",Landscape alterations and stair revision,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix)


In [154]:
df['project_location_clean'] = df['project_location'].apply(
    lambda x: ' '.join(
        [location.lower().replace(' - ','-').strip() for location in x if location.lower() != "united states"]
        )
    ).str.replace(', ',' ').str.replace(' washington dc','').str.replace(' st.',' street').str.replace('½','1/2').str.replace('−','-')

df.drop(columns=['project_location'], inplace=True)

In [155]:
og = df[df['project_number'].apply(lambda x: x.split()[0] == 'OG')]

#### Check for Duplicate Addresses per Project

Are there any duplicate project locations per project number?

In [159]:
duplicate_addresses = og.groupby(['project_number'])['project_location_clean'].unique().apply(len).sort_values(ascending=False)
duplicate_addresses = duplicate_addresses[duplicate_addresses > 1].index
len(duplicate_addresses)

32

Seeing which ones have multiple addresses associated with the project

In [160]:
for a in duplicate_addresses:
    print(a)
    print(og[og['project_number'] == a]['project_location_clean'].unique())
    print('\n')

OG 24-163
['1805 wisconsin avenue nw' '1803 wisconsin avenue nw']


OG 12-337
['3220 grace street nw'
 '3220 grace street nw (also known as 3240 grace street nw)']


OG 15-165
['3271-3273 m street nw' '3271 and 3273 m street nw']


OG 24-194
['adjacent to 3419 r street nw' 'tbd']


OG 24-253
['tbd (adjacent to 3419 r street nw)' 'tbd']


OG 15-170
['2522 q street nw' '2722 q street nw']


OG 18-100
['3700 o street nw' '3700 o street nw-white gravenor hall']


OG 23-317
['1312 30th street nw' '2915 q street nw']


OG 07-127
['1645 31st street nw' 'single-family residence 1645 31st street nw']


OG 14-369
['3252 jones court nw' '3252 jones court']


OG 12-145
['3138 p street nw' '13138 p street nw']


OG 16-194
['1301-29 37th street 3614-22 o street 3615-25 n street nw'
 '1301-1329 37th 3614-3622 o 3615-3625 n streets nw']


OG 22-177
['3299 k (water) street nw' '3299 water street nw']


OG 22-069
['3401 water street nw' '2909 m street nw']


OG 14-235
['3330 m street nw (rear)' '3330 m 

#### Map the Locations

Convert the projects into project locations 

In [167]:
og.head(3)

,project_number,project_link,project_type,project_description,last_reviewed,last_reviewed_meeting,review_type,project_location_clean
0,OG 25-038,/records-research/project-search/og-25-038-0,Commercial,Sign - Canal House,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1023-1029 31st street nw
1,OG 25-037,/records-research/project-search/og-25-037-0,Georgetown,Shaw Field lights,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3700 o street nw
2,OG 25-036,/records-research/project-search/og-25-036-0,Residence,Egress window well on west elevation,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3321 q street nw


In [180]:
og[og['project_location_clean'].str.contains('-')]

,project_number,project_link,project_type,project_description,last_reviewed,last_reviewed_meeting,review_type,project_location_clean
0,OG 25-038,/records-research/project-search/og-25-038-0,Commercial,Sign - Canal House,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1023-1029 31st street nw
25,OG 24-361,/records-research/project-search/og-24-361-1,Commercial,"New storefront, signage, entrances, and painti...",2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),3233-3235 m street nw
28,OG 24-342,/records-research/project-search/og-24-342-1,Commercial,Signage - Canal House,2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1023-29 31st street nw
30,OG 24-337,/records-research/project-search/og-24-337-0,Commercial,"New gate, fenestration alterations, sign",2024-11-21T12:00:00Z,/records-research/record-cfa-actions/2024/11/c...,(CFA Meeting - OG Appendix),1220-1236 31st street nw
94,OG 25-038,/records-research/project-search/og-25-038,Commercial,Signage,2024-11-07T12:00:00Z,/records-research/record-cfa-actions/2024/11/o...,(OGB Meeting - Consent Calendar),1023-1029 31st street nw
...,...,...,...,...,...,...,...,...
18569,OG 03-142,/records-research/project-search/og-03-142,Location1235-37,Replacement windows in front facade,2003-04-22T12:00:00Z,/records-research/record-cfa-actions/2003/04/c...,(CFA Meeting - OG Appendix),1235-37 wisconsin avenue nw
18573,OG 03-138,/records-research/project-search/og-03-138,Eagle,Alterations to storefronts for new building-re...,2003-04-22T12:00:00Z,/records-research/record-cfa-actions/2003/04/c...,(CFA Meeting - OG Appendix),3331-47 m street nw
18712,OG 03-051,/records-research/project-search/og-03-051,Residence,New board fences and repair of gate,2003-01-16T12:00:00Z,/records-research/record-cfa-actions/2003/01/c...,(CFA Meeting - OG Appendix),1528 -30 -32 34th street nw
18715,OG 03-034,/records-research/project-search/og-03-034,Eagle,Alterations to storefronts for new building,2003-01-16T12:00:00Z,/records-research/record-cfa-actions/2003/01/c...,(CFA Meeting - OG Appendix),3331-3347 m street nw


In [196]:
def clean_dashes_address(address):
    # Remove digits with a dash in front of them
    cleaned_address = re.sub(r'-\d+', '', address)
    # Remove a dash followed by a single letter and a space
    cleaned_address = re.sub(r'-[a-zA-Z] ', '', cleaned_address)
    # Remove any extra spaces
    cleaned_address = re.sub(r'\s+', ' ', cleaned_address).strip()

    # Find the index of the first digit in the address
    first_digit_index = re.search(r'\d', cleaned_address)
    if first_digit_index:
        cleaned_address = cleaned_address[first_digit_index.start():]
    return cleaned_address

In [ ]:
og['project_location_preload_geolocator'] = og['project_location_clean'].apply(clean_dashes_address)
og['project_location_preload_geolocator'] = og['project_location_preload_geolocator'] + ", washington, district of columbia"

In [27]:
# Initialize geocoder
geolocator = Nominatim(user_agent="geoapi")

def get_coordinates(address):
    try:
        # Geocode the address
        location = geolocator.geocode(address, timeout=15)
        if location:
            return location
        else:
            return None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None

In [ ]:
#100m 46.1s
for i in range(0, len(og), 100):
    print(f"Processing rows {i} to {i+100}")
    og.loc[i:i+100,'Coordinates'] = og['project_location_preload_geolocator'].iloc[i:i+100].apply(get_coordinates)

Processing rows 0 to 100


C:\Users\finnr\AppData\Local\Temp\ipykernel_23496\143738719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og.loc[i:i+100,'Coordinates'] = og['project_location_preload_geolocator'].iloc[i:i+100].apply(get_coordinates)


Processing rows 100 to 200
Processing rows 200 to 300
Processing rows 300 to 400
Processing rows 400 to 500
Processing rows 500 to 600
Processing rows 600 to 700
Processing rows 700 to 800
Processing rows 800 to 900
Processing rows 900 to 1000
Processing rows 1000 to 1100
Processing rows 1100 to 1200
Processing rows 1200 to 1300
Processing rows 1300 to 1400
Processing rows 1400 to 1500
Processing rows 1500 to 1600
Processing rows 1600 to 1700
Processing rows 1700 to 1800
Processing rows 1800 to 1900
Processing rows 1900 to 2000
Processing rows 2000 to 2100
Processing rows 2100 to 2200
Processing rows 2200 to 2300
Processing rows 2300 to 2400
Processing rows 2400 to 2500
Processing rows 2500 to 2600
Processing rows 2600 to 2700
Processing rows 2700 to 2800
Processing rows 2800 to 2900
Processing rows 2900 to 3000
Processing rows 3000 to 3100
Processing rows 3100 to 3200
Processing rows 3200 to 3300
Processing rows 3300 to 3400
Processing rows 3400 to 3500
Processing rows 3500 to 3600
Pr

In [28]:
t = og.fillna('project_location_preload_geolocator')['Coordinates']
for i in range(0, len(og), 100):
    print(f"Processing rows {i} to {i+100}")
    og.iloc[i:i+100,-1] = t.iloc[i:i+100].apply(get_coordinates)

Processing rows 0 to 100
Processing rows 100 to 200
Processing rows 200 to 300
Processing rows 300 to 400
Processing rows 400 to 500
Processing rows 500 to 600
Processing rows 600 to 700
Processing rows 700 to 800
Processing rows 800 to 900
Processing rows 900 to 1000
Processing rows 1000 to 1100
Processing rows 1100 to 1200
Processing rows 1200 to 1300
Processing rows 1300 to 1400
Processing rows 1400 to 1500
Processing rows 1500 to 1600
Processing rows 1600 to 1700
Processing rows 1700 to 1800
Processing rows 1800 to 1900
Processing rows 1900 to 2000
Processing rows 2000 to 2100
Processing rows 2100 to 2200
Processing rows 2200 to 2300
Processing rows 2300 to 2400
Processing rows 2400 to 2500
Processing rows 2500 to 2600
Processing rows 2600 to 2700
Processing rows 2700 to 2800
Processing rows 2800 to 2900
Processing rows 2900 to 3000
Processing rows 3000 to 3100
Processing rows 3100 to 3200
Processing rows 3200 to 3300
Processing rows 3300 to 3400
Processing rows 3400 to 3500
Proces

In [263]:
for i in range(700, len(og), 100):
    print(f"Processing rows {i} to {i+100}")
    og.iloc[i:i+100,-1] = og['project_location_preload_geolocator'].iloc[i:i+100].apply(get_coordinates)

Processing rows 700 to 800
Processing rows 800 to 900
Processing rows 900 to 1000
Processing rows 1000 to 1100
Processing rows 1100 to 1200
Processing rows 1200 to 1300
Processing rows 1300 to 1400
Processing rows 1400 to 1500
Processing rows 1500 to 1600
Processing rows 1600 to 1700
Processing rows 1700 to 1800
Processing rows 1800 to 1900
Processing rows 1900 to 2000
Processing rows 2000 to 2100
Processing rows 2100 to 2200
Processing rows 2200 to 2300
Processing rows 2300 to 2400
Processing rows 2400 to 2500
Processing rows 2500 to 2600
Processing rows 2600 to 2700
Processing rows 2700 to 2800
Processing rows 2800 to 2900
Processing rows 2900 to 3000
Processing rows 3000 to 3100
Processing rows 3100 to 3200
Processing rows 3200 to 3300
Processing rows 3300 to 3400
Processing rows 3400 to 3500
Processing rows 3500 to 3600
Processing rows 3600 to 3700
Processing rows 3700 to 3800
Processing rows 3800 to 3900
Processing rows 3900 to 4000
Processing rows 4000 to 4100
Processing rows 410

In [ ]:
og['Coordinates_lat_lon'] = og['Coordinates'].apply(lambda x: (x.latitude, x.longitude) if x else None)

In [61]:
og.to_csv('../Data_Raw/project_tables/og_geocoded.csv', index=False)